# Лабораторная работа №2

## Подключение библиотек

Подключим сторонние библиотеки:

- `numpy` для удобной работы с данными
- `matplotlib` для рисование графиков

In [2]:
import numpy as np
import matplotlib.pyplot as plt

Также подключим некоторые стандартные библиотеки

In [42]:
from pprint import pprint
from collections import Counter
from typing import List, Union, Tuple

## Дополнительно 

Создадим функцию для более короткого представления массива из элементов np.float

In [43]:
def arr_to_str(arr: List[Union[np.float64, Tuple[np.float64, np.float64]]], digits: int = 4) -> str:
    def round_value(x):
        return round(float(x), digits)
    
    if arr and isinstance(arr[0], tuple):
        return str([(round_value(x), round_value(y)) for x, y in arr])
    else:
        return str([round_value(x) for x in arr])

## Импорт данных

Загрузим **файлы данных** и проверим корректно ли они загрузились

In [5]:
data_sets = []
for i in range(3):
    filename = f'set_{i + 1}.csv'
    try:
        cur_data = np.loadtxt(filename, delimiter=',')
        data_sets.append(cur_data)
    except FileNotFoundError:
        print(f"Ошибка: Файл {filename} не найден")
    except ValueError:
        print(f"Ошибка: Некорректный формат данных в файле {filename}")

SIZE = len(data_sets)
pprint(data_sets)

[array([3., 1., 1., ..., 1., 1., 3.], shape=(100000,)),
 array([ 90.547448,  80.548716,  92.992958, ...,   9.088514, 328.988096,
        84.872411], shape=(10000,)),
 array([0.002163, 0.023507, 0.067606, ..., 0.109763, 0.041012, 0.040421],
      shape=(100000,))]


## Анализ данных

Выборочное математическое ожидание можем найти по следующей формуле:

$$\bar{x}_{B}=\frac{x_1+x_2+\ldots+x_n}{n}$$

А несмещенную (исправленную) дисперсию по следующей формуле:

$$s^2=\frac{1}{n-1}\sum_{i=1}^{n} (x_i - \bar{x}_{B})^2$$

In [6]:
def calculate_mean(data: np.ndarray) -> np.float64:    
    return sum(data) / len(data)


def calculate_unbiased_variance(data: np.ndarray, mean: np.float64 | None = None) -> np.float64:
    if mean is None:
        mean = calculate_mean(data)
    return 1 / (len(data) - 1) * sum((data[i] - mean) ** 2 for i in range(len(data))) 

Посчитаем $\bar{x}_B$, а также $s^2$ для каждого набора данных

In [7]:
means = [calculate_mean(data_sets[i]) for i in range(SIZE)]
unbiased_variances = [calculate_unbiased_variance(data_sets[i], means[i]) for i in range(SIZE)]

print("Выборочное мат. ожидание:", arr_to_str(means))
print("Выборочная несмещенная дисперсия:", arr_to_str(unbiased_variances))

Выборочное мат. ожидание: [3.4408, 16.2433, 0.1245]
Выборочная несмещенная дисперсия: [10.0483, 10928.4975, 0.0256]


## Построение доверительных интервалов

Сначала нужно построить таблицу критических точек распределения Стьюдента ($t$-распределения). Построю её, например, для уровня значимости $a=0.1$

In [ ]:
def _get_t_table():
    t_table = {
        1: 6.31, 2: 2.92, 3: 2.35, 4: 2.13, 5: 2.01,
        6: 1.94, 7: 1.89, 8: 1.86, 9: 1.83, 10: 1.81,
        11: 1.80, 12: 1.78, 13: 1.77, 14: 1.76, 15: 1.75,
        16: 1.75, 17: 1.74, 18: 1.73, 19: 1.73, 20: 1.73,
        21: 1.72, 22: 1.72, 23: 1.71, 24: 1.71, 25: 1.71,
        26: 1.71, 27: 1.71, 28: 1.70, 29: 1.70, 30: 1.70,
        40: 1.68, 60: 1.67, 120: 1.66, float('inf'): 1.64,
    }

    def wrapper(n: int) -> float:
        if n in t_table:
            return t_table[n]
        
        if n > 120:
            return 1.66 - (1.66 - 1.64) * min((n - 120) / 10000, 1)
        
        lower_n = max(k for k in t_table if k < n)
        upper_n = min(k for k in t_table if k > n)
        
        lower_val = t_table[lower_n]
        upper_val = t_table[upper_n]
        
        ratio = (n - lower_n) / (upper_n - lower_n)
        return lower_val + ratio * (upper_val - lower_val)
    return wrapper


t = _get_t_table()

Теперь найдем доверительный интервал для математического ожидания, в предположении, что у нас данные подчиняются нормальному распределению.

Искомый интервал $(a, b)$ ищется по формулам:
$$a=\bar{x}_B - \frac{t_{n-1} \cdot s}{\sqrt{n}}$$

$$b=\bar{x}_B + \frac{t_{n-1} \cdot s}{\sqrt{n}}$$

In [39]:
def get_mean_interval(mean: np.float64, s: np.float64, n: int):
    part = t(n - 1) * s / np.sqrt(n)
    return ((mean - part), (mean + part))

Рассчитаем искомые интервалы математического ожидания для каждого набора данных.

In [47]:
mean_intervals = [
    get_mean_interval(means[i], np.sqrt(unbiased_variances[i]), len(data_sets[i]))
    for i in range(SIZE)
]

print(arr_to_str(mean_intervals))

[(3.4243, 3.4572), (14.5286, 17.958), (0.1237, 0.1253)]


Скопирую функцию, использованную в 1 лабораторной работе, для генерации нормальных случайных величин.

In [ ]:
def box_muller():
    u1 = np.random.rand()
    u2 = np.random.rand()
    
    r = np.sqrt(-2 * np.log(1 - u1))
    theta = 2 * np.pi * u2

    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y


def normal_distribution(mean: float, variance: float, size: int) -> np.ndarray:
    std_dev = np.sqrt(variance)
    random_numbers = np.zeros(size)

    for i in range(size // 2):
        x, y = box_muller()

        random_numbers[2*i] = x * std_dev + mean
        random_numbers[2*i + 1] = y * std_dev + mean
    
    if size % 2 != 0:
        x, y = box_muller()
        random_numbers[-1] = x * std_dev + mean
    return random_numbers
